In [3]:
pip install "numpy<2.0.0"


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import fitz  # PyMuPDF
import re
import csv
import os

C:\Users\aryan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\aryan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
aw_template_path = "NewBaltimo 532025 to 5162025.csv"
aw_df = pd.read_csv(aw_template_path, skiprows=9)

# Load the original file with headers (to combine header rows 8 and 9)
with open(aw_template_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Extract header rows (row 8 and 9)
header_line_1 = lines[7].strip().split(",")
header_line_2 = lines[8].strip().split(",")

# Combine the two header lines into one
combined_headers = [
    (h1.strip() + "_" + h2.strip()) if h1.strip() and h2.strip() else (h1.strip() or h2.strip())
    for h1, h2 in zip(header_line_1, header_line_2)
]

# Apply combined headers to the data
aw_df.columns = combined_headers

aw_df

,Emp,Employee,S/H_Salaried,DEPT_Dept,RegHrs_Regular,RegAmt_Regular,VacHrs_Vacation,VacAmt_Vacation,HolHrs_Holiday,HolAmt_Holiday,...,DED514794_Dental Insurance,DED514795_Loan Repayment,DED514800_Medical Ins,DED514798_Medical Insurance,DED514792_Pre Tax SCP,DED514801_Roth 457(b) (EE),DED514802_Union Dues,DED514799_Vision Ins,DED514803_Vision Insurance,NetAmt_Net
0,74.0,Albano Craig J.,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,152.0,Anderson Samuel T.,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1.0,Bennett Gordon W.,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,121.0,Besenfelder Kathryn E.,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,4.0,Betke Louis P.,H,Sewer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,107.0,Varade David M.,H,Sewer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
90,134.0,Vieta Sherry L.,S,General,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
91,141.0,Woelfersheim Joseph A.,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
92,111.0,Woodbeck Herbert D.,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,Payroll Register Report,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,Town of New Baltimore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Pay Period From 05/03/2025 to 05/16/2025, Pay ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Earnings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hours*,NaN,...,YTD,NaN,ER Taxes**,NaN,NaN,NaN,NaN,Current,NaN,YTD
3,NaN,Department :,NaN,NaN,NaN,General,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Emp# 137,NaN,NaN,NaN,NaN,Dawn M DeRose,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import pandas as pd
import json

# Load the uploaded CSV file
csv_path = "NewBaltimo 532025 to 5162025.csv"
df = pd.read_csv(csv_path)

# Convert DataFrame to JSON
json_data = df.to_dict(orient='records')

# Save the JSON data to a file
json_data


[{'Firm:': 'Employer:',
  'Unnamed: 1': nan,
  'Evening Star Bookkeeping Service Inc': 'Town of New Baltimore',
  'Unnamed: 3': nan,
  'Unnamed: 4': nan,
  'Firm Code:': 'Client Code:',
  'Unnamed: 6': nan,
  'STARB3966': 'NewBaltimo',
  'Unnamed: 8': nan,
  'Unnamed: 9': nan,
  'Unnamed: 10': nan,
  'Unnamed: 11': nan,
  'Unnamed: 12': nan,
  'Unnamed: 13': nan,
  'Unnamed: 14': nan,
  'Unnamed: 15': nan,
  'Unnamed: 16': nan,
  'Unnamed: 17': nan,
  'Unnamed: 18': nan,
  'Unnamed: 19': nan,
  'Unnamed: 20': nan,
  'Unnamed: 21': nan,
  'Unnamed: 22': nan,
  'Unnamed: 23': nan,
  'Unnamed: 24': nan,
  'Unnamed: 25': nan,
  'Unnamed: 26': nan,
  'Unnamed: 27': nan,
  'Unnamed: 28': nan,
  'Unnamed: 29': nan,
  'Unnamed: 30': nan,
  'Unnamed: 31': nan,
  'Unnamed: 32': nan,
  'Unnamed: 33': nan,
  'Unnamed: 34': nan,
  'Unnamed: 35': nan,
  'Unnamed: 36': nan,
  'Unnamed: 37': nan,
  'Unnamed: 38': nan,
  'Unnamed: 39': nan,
  'Unnamed: 40': nan,
  'Unnamed: 41': nan,
  'Unnamed: 42': n

In [20]:
# Reload the DataFrame for clean reprocessing
df = pd.read_csv(csv_path, header=None)

# Step 1: Find the header row index (the row that defines column names)
# We assume the row starting with 'Emp' is the header we care about
header_row_index = df[df.eq("Emp").any(axis=1)].index[0]

# Step 2: Set the proper header and drop rows above it
df.columns = df.iloc[header_row_index]
df = df.drop(index=range(0, header_row_index + 1)).reset_index(drop=True)

# Step 3: Drop columns that are entirely NaN (optional)
df = df.dropna(axis=1, how='all')

# Step 4: Convert to JSON
structured_json = df.to_dict(orient='records')

# Save cleaned JSON
structured_json

C:\Users\aryan\AppData\Local\Temp\ipykernel_33336\4077635703.py:16: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  structured_json = df.to_dict(orient='records')


[{'Emp': 'Num',
  'Employee': 'Name',
  'Salaried': '/ Hourly',
  'Dept': 'Code',
  'Regular': 'Amount',
  'Vacation': 'Amount',
  'Holiday': 'Amount',
  'Reimbursement': 'Amount',
  'Overtime': 'Amount',
  'Sick': 'Amount',
  'Personal': 'Amount',
  'Deputy Clerk 1410': 'Amount',
  'Records 1460': 'Amount',
  'Comp Time': 'Amount',
  'Office worker': 'Amount',
  'Jury Duty': 'Amount',
  'Emergency Mgmt': 'Amount',
  'Bereavment': 'Amount',
  'Retro Wages': 'Amount',
  'Deputy Supt': 'Amount',
  'Supv Secretary': 'Amount',
  'Assessor 1355': 'Amount',
  'Codes 3620': 'Amount',
  'Zoning 8010': 'Amount',
  'Planning 8020': 'Amount',
  'Collector 1330': 'Amount',
  'Federal': 'Tax',
  'Soc.Sec.': 'Tax',
  'Medicare': 'Tax',
  'NY State': 'Tax',
  'NY PFML': 'Tax',
  'NY SDI': 'Tax',
  '403(b) for EE 50+ (E)': 'Amount',
  '414(H)': 'Amount',
  '457(b)': 'Amount',
  '457(b) (50+)': 'Amount',
  'Aflac': 'Amount',
  'Aflac Pre-Tax': 'Amount',
  'Child/Spousal Support': 'Amount',
  'Colonial 

In [21]:
import numpy as np

# Reload CSV without header to process manually
raw_df = pd.read_csv(csv_path, header=None)

# Step 1: Identify start of employee blocks by "Emp#" keyword
employee_rows = raw_df[raw_df.apply(lambda row: row.astype(str).str.contains("Emp#").any(), axis=1)]

# Store structured employee data here
employee_data = []

for idx in employee_rows.index:
    # Extract top-level metadata
    emp_info = raw_df.iloc[idx]
    emp_number = emp_info[0].split("#")[1].strip() if "#" in emp_info[0] else None
    emp_name = emp_info[1]
    
    # Search downward from current row for relevant info
    block = raw_df.iloc[idx:idx+10].reset_index(drop=True)
    emp_dict = {
        "Emp#": emp_number,
        "Name": emp_name,
        "Department": str(block.iloc[0][3]) if pd.notna(block.iloc[0][3]) else None,
        "Net Pay": block.iloc[1][4] if pd.notna(block.iloc[1][4]) else None,
        "Hours": float(block.iloc[2][0]) if pd.notna(block.iloc[2][0]) else None,
        "Rate": float(block.iloc[2][1]) if pd.notna(block.iloc[2][1]) else None,
        "Earnings": {
            "Current": float(block.iloc[2][2]) if pd.notna(block.iloc[2][2]) else None,
            "YTD": float(block.iloc[2][3]) if pd.notna(block.iloc[2][3]) else None
        },
        "Taxes": {},
        "ER Taxes": {},
        "Employee Totals": {}
    }

    # Parse Taxes
    for i in range(3, 7):
        tax_line = block.iloc[i]
        tax_type = str(tax_line[4]) if pd.notna(tax_line[4]) else None
        if tax_type in ["FWT", "SS W/H", "MC W/H", "NY State Tax", "NY SDI"]:
            emp_dict["Taxes"][tax_type] = {
                "Current": float(tax_line[5]) if pd.notna(tax_line[5]) else 0.0,
                "YTD": float(tax_line[6]) if pd.notna(tax_line[6]) else 0.0
            }

    # Parse ER Taxes
    for i in range(3, 6):
        er_tax_line = block.iloc[i]
        er_type = str(er_tax_line[7]) if pd.notna(er_tax_line[7]) else None
        if er_type in ["ER SS", "ER MC"]:
            emp_dict["ER Taxes"][er_type] = {
                "Current": float(er_tax_line[8]) if pd.notna(er_tax_line[8]) else 0.0,
                "YTD": float(er_tax_line[9]) if pd.notna(er_tax_line[9]) else 0.0
            }

    # Parse Employee Totals
    totals_line = block[block.apply(lambda row: row.astype(str).str.contains("Employee Tot", na=False).any(), axis=1)]
    if not totals_line.empty:
        t = totals_line.iloc[0]
        emp_dict["Employee Totals"] = {
            "Earnings": {"Current": float(t[4]) if pd.notna(t[4]) else 0.0, "YTD": float(t[5]) if pd.notna(t[5]) else 0.0},
            "Taxes": {"Current": float(t[6]) if pd.notna(t[6]) else 0.0, "YTD": float(t[7]) if pd.notna(t[7]) else 0.0},
            "Deductions": {"Current": float(t[8]) if pd.notna(t[8]) else 0.0, "YTD": float(t[9]) if pd.notna(t[9]) else 0.0},
            "ER Taxes": {"Current": float(t[10]) if pd.notna(t[10]) else 0.0, "YTD": float(t[11]) if pd.notna(t[11]) else 0.0}
        }

    employee_data.append(emp_dict)

# Save as JSON
employee_data

[]

In [24]:
# Use the new uploaded CSV file for processing
new_csv_path = "NewBaltimo 532025 to 5162025.csv"
raw_df_new = pd.read_csv(new_csv_path, header=bNone,skiprows=10)

# Reuse the same logic on the new file
employee_rows_new = raw_df_new[raw_df_new.apply(lambda row: row.astype(str).str.contains("Emp#").any(), axis=1)]
employee_data_new = []

for idx in employee_rows_new.index:
    emp_info = raw_df_new.iloc[idx]
    emp_number = emp_info[0].split("#")[1].strip() if "#" in emp_info[0] else None
    emp_name = emp_info[1]

    block = raw_df_new.iloc[idx:idx+10].reset_index(drop=True)
    emp_dict = {
        "Emp#": emp_number,
        "Name": emp_name,
        "Department": str(block.iloc[0][3]) if pd.notna(block.iloc[0][3]) else None,
        "Net Pay": block.iloc[1][4] if pd.notna(block.iloc[1][4]) else None,
        "Hours": float(block.iloc[2][0]) if pd.notna(block.iloc[2][0]) else None,
        "Rate": float(block.iloc[2][1]) if pd.notna(block.iloc[2][1]) else None,
        "Earnings": {
            "Current": float(block.iloc[2][2]) if pd.notna(block.iloc[2][2]) else None,
            "YTD": float(block.iloc[2][3]) if pd.notna(block.iloc[2][3]) else None
        },
        "Taxes": {},
        "ER Taxes": {},
        "Employee Totals": {}
    }

    for i in range(3, 7):
        tax_line = block.iloc[i]
        tax_type = str(tax_line[4]) if pd.notna(tax_line[4]) else None
        if tax_type in ["FWT", "SS W/H", "MC W/H", "NY State Tax", "NY SDI"]:
            emp_dict["Taxes"][tax_type] = {
                "Current": float(tax_line[5]) if pd.notna(tax_line[5]) else 0.0,
                "YTD": float(tax_line[6]) if pd.notna(tax_line[6]) else 0.0
            }

    for i in range(3, 6):
        er_tax_line = block.iloc[i]
        er_type = str(er_tax_line[7]) if pd.notna(er_tax_line[7]) else None
        if er_type in ["ER SS", "ER MC"]:
            emp_dict["ER Taxes"][er_type] = {
                "Current": float(er_tax_line[8]) if pd.notna(er_tax_line[8]) else 0.0,
                "YTD": float(er_tax_line[9]) if pd.notna(er_tax_line[9]) else 0.0
            }

    totals_line = block[block.apply(lambda row: row.astype(str).str.contains("Employee Tot", na=False).any(), axis=1)]
    if not totals_line.empty:
        t = totals_line.iloc[0]
        emp_dict["Employee Totals"] = {
            "Earnings": {"Current": float(t[4]) if pd.notna(t[4]) else 0.0, "YTD": float(t[5]) if pd.notna(t[5]) else 0.0},
            "Taxes": {"Current": float(t[6]) if pd.notna(t[6]) else 0.0, "YTD": float(t[7]) if pd.notna(t[7]) else 0.0},
            "Deductions": {"Current": float(t[8]) if pd.notna(t[8]) else 0.0, "YTD": float(t[9]) if pd.notna(t[9]) else 0.0},
            "ER Taxes": {"Current": float(t[10]) if pd.notna(t[10]) else 0.0, "YTD": float(t[11]) if pd.notna(t[11]) else 0.0}
        }

    employee_data_new.append(emp_dict)

# Save as JSON
employee_data_new


[]

In [28]:
import pandas as pd
import re
import json

def parse_payroll_excel(filepath):
    df = pd.read_excel(filepath, header=None)
    
    employees = []
    for i in range(len(df)):
        row = df.iloc[i].astype(str).tolist()

        if any("Emp#" in str(cell) for cell in row):
            emp_block = df.iloc[i:i+10].reset_index(drop=True)
            
            emp_number = re.search(r"Emp#\s*(\d+)", str(emp_block.iloc[0, 0]))
            name = str(emp_block.iloc[0, 1]).strip()
            net_pay = re.search(r"Net Pay:\s*(\d+\.\d+)", str(emp_block.iloc[1, 4]))

            emp_data = {
                "Emp#": emp_number.group(1) if emp_number else None,
                "Name": name,
                "Hours": float(emp_block.iloc[2, 0]) if pd.notna(emp_block.iloc[2, 0]) else None,
                "Rate": float(emp_block.iloc[2, 1]) if pd.notna(emp_block.iloc[2, 1]) else None,
                "Current Pay": float(emp_block.iloc[2, 3]) if pd.notna(emp_block.iloc[2, 3]) else None,
                "YTD Pay": float(emp_block.iloc[2, 4]) if pd.notna(emp_block.iloc[2, 4]) else None,
                "Net Pay": float(net_pay.group(1)) if net_pay else None,
                "Taxes": {
                    "FWT": float(emp_block.iloc[3, 5]) if pd.notna(emp_block.iloc[3, 5]) else None,
                    "SS W/H": float(emp_block.iloc[4, 5]) if pd.notna(emp_block.iloc[4, 5]) else None,
                    "MC W/H": float(emp_block.iloc[5, 5]) if pd.notna(emp_block.iloc[5, 5]) else None,
                    "NY State Tax": float(emp_block.iloc[6, 5]) if pd.notna(emp_block.iloc[6, 5]) else None,
                    "NY SDI": float(emp_block.iloc[7, 5]) if pd.notna(emp_block.iloc[7, 5]) else None,
                },
                "ER Taxes": {
                    "ER SS": float(emp_block.iloc[3, 10]) if pd.notna(emp_block.iloc[3, 10]) else None,
                    "ER MC": float(emp_block.iloc[4, 10]) if pd.notna(emp_block.iloc[4, 10]) else None,
                }
            }
            employees.append(emp_data)

    return employees

# Example usage
filepath = "Book1.xlsx"
parsed_data = parse_payroll_excel(filepath)

# Save as JSON
with open("parsed_payroll.json", "w") as f:
    json.dump(parsed_data, f, indent=2)


In [33]:
payroll_path='payroll register report 5325 to 51625.xlsx'

In [31]:
payroll=pd.read_excel('payroll register report 5325 to 51625.xlsx',header=None)
payroll

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,Payroll Register Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Town of New Baltimore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Pay Period From 05/03/2025 to 05/16/2025, Pay ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Earnings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hours*,NaN,...,YTD,NaN,ER Taxes**,NaN,NaN,NaN,NaN,Current,NaN,YTD
4,NaN,Department :,NaN,NaN,NaN,General,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
from openpyxl import load_workbook

# Load workbook and sheet
wb = load_workbook(payroll_path, data_only=True)
sheet = wb.active

# Iterate through rows and filter out None values
cleaned_rows = []
for row in sheet.iter_rows(values_only=True):
    non_none_values = [cell for cell in row if cell is not None]
    if non_none_values:
        cleaned_rows.append(non_none_values)



In [43]:
cleaned_rows

[['Payroll Register Report'],
 ['Town of New Baltimore'],
 ['Pay Period From 05/03/2025 to 05/16/2025, Pay Date: 05/23/2025, Payroll # 13 (Standard)'],
 ['Earnings',
  'Hours*',
  'Rate',
  'Current',
  'YTD',
  'Taxes',
  'Current',
  'YTD',
  'Deductions',
  'Current',
  'YTD',
  'ER Taxes**',
  'Current',
  'YTD'],
 ['Department :', 'General '],
 ['Emp# 137', 'Dawn M DeRose'],
 ['DirDep', 'Net Pay:', 554.07],
 ['10.00',
  '74.9192',
  '749.19',
  '8,219.29',
  'FWT',
  '117.23',
  '1,288.89',
  'ER SS',
  '46.45',
  '509.60'],
 ['Regular', 'SS W/H', '46.45', '509.60', 'ER MC', '10.86', '119.18'],
 ['MC W/H', '10.86', '119.18'],
 ['NY State Tax', '19.38', '212.09'],
 ['NY SDI', '1.20', '13.20'],
 ['Employee Tot:', 10, 749.19, 8219.29, 195.12, 2142.96, 0, 0, 57.31, 628.78],
 ['Emp# 160', 'Jessica M Diamond'],
 ['DirDep', 'Net Pay:', 1099.96],
 ['65.00',
  '20.57',
  '1,337.05',
  '2,704.96',
  'FWT',
  '82.04',
  '135.70',
  'ER SS',
  '82.90',
  '167.71'],
 ['Regular', 'SS W/H', '82.

In [41]:
cleaned_rows_split = []

for row in sheet.iter_rows(values_only=True):
    # Filter out None values
    non_none = [cell for cell in row if cell is not None]
    if not non_none:
        continue

    # Split strings by comma and keep numbers as-is
    split_row = []
    for cell in non_none:
        if isinstance(cell, str):
            split_row.extend([item.strip() for item in cell.split(',')])
        else:
            split_row.append(cell)

    cleaned_rows_split.append(split_row)

cleaned_rows_split


[['Payroll Register Report'],
 ['Town of New Baltimore'],
 ['Pay Period From 05/03/2025 to 05/16/2025',
  'Pay Date: 05/23/2025',
  'Payroll # 13 (Standard)'],
 ['Earnings',
  'Hours*',
  'Rate',
  'Current',
  'YTD',
  'Taxes',
  'Current',
  'YTD',
  'Deductions',
  'Current',
  'YTD',
  'ER Taxes**',
  'Current',
  'YTD'],
 ['Department :', 'General'],
 ['Emp# 137', 'Dawn M DeRose'],
 ['DirDep', 'Net Pay:', 554.07],
 ['10.00',
  '74.9192',
  '749.19',
  '8',
  '219.29',
  'FWT',
  '117.23',
  '1',
  '288.89',
  'ER SS',
  '46.45',
  '509.60'],
 ['Regular', 'SS W/H', '46.45', '509.60', 'ER MC', '10.86', '119.18'],
 ['MC W/H', '10.86', '119.18'],
 ['NY State Tax', '19.38', '212.09'],
 ['NY SDI', '1.20', '13.20'],
 ['Employee Tot:', 10, 749.19, 8219.29, 195.12, 2142.96, 0, 0, 57.31, 628.78],
 ['Emp# 160', 'Jessica M Diamond'],
 ['DirDep', 'Net Pay:', 1099.96],
 ['65.00',
  '20.57',
  '1',
  '337.05',
  '2',
  '704.96',
  'FWT',
  '82.04',
  '135.70',
  'ER SS',
  '82.90',
  '167.71'],


In [55]:
from openpyxl import load_workbook

# Load workbook
wb = load_workbook(payroll_path, data_only=True)
sheet = wb.active

data = []
start_reading = False
header_rows = []

for row in sheet.iter_rows(values_only=True):
    if not start_reading:
        if row and any(str(cell).strip() == 'Emp#' for cell in row):
            # Found the start of the structured table
            header_rows.append([str(cell).strip() if cell else '' for cell in row])
            start_reading = True
        continue

    # Read the second header row after 'Emp Num'
    if len(header_rows) == 1:
        header_rows.append([str(cell).strip() if cell else '' for cell in row])
        # Merge headers
        headers = []
        for h1, h2 in zip(header_rows[0], header_rows[1]):
            merged = (h1 + ' ' + h2).strip()
            headers.append(merged if merged else h1 or h2)
        continue

    # Read employee rows
    row_values = [cell for cell in row[:len(headers)]]
    data.append(dict(zip(headers, row_values)))

# Example: show the first 2 employee records
for record in data[:2]:
    print(record)
    print('p')


In [56]:
data

[]

In [57]:
from openpyxl import load_workbook

# Load workbook
wb = load_workbook(payroll_path, data_only=True)
sheet = wb.active

data = []
employee_blocks = []

rows = list(sheet.iter_rows(values_only=True))
num_rows = len(rows)

# Loop to detect blocks that start with "Emp# <number>"
i = 0
while i < num_rows:
    row = rows[i]
    if row and any(str(cell).strip().startswith("Emp# ") for cell in row if cell):
        # Start of employee block
        emp_block = []
        emp_block.append([str(cell).strip() if cell else '' for cell in row])

        # Collect next 10–15 rows (until another Emp# or empty break)
        j = i + 1
        while j < num_rows:
            next_row = rows[j]
            if next_row and any(str(cell).strip().startswith("Emp# ") for cell in next_row if cell):
                break  # start of next employee
            if next_row and any(cell is not None and str(cell).strip() != '' for cell in next_row):
                emp_block.append([str(cell).strip() if cell else '' for cell in next_row])
            else:
                break  # likely end of block
            j += 1

        employee_blocks.append(emp_block)
        i = j
    else:
        i += 1

# Example: show first 2 employee blocks
for block in employee_blocks[:2]:
    for row in block:
        print(row)
    print("------")


['', 'Emp# 137', '', '', '', '', 'Dawn M DeRose', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', 'DirDep', '', '', '', '', '', '', 'Net Pay:', '', '554.07', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '10.00', '', '74.9192', '', '', '', '749.19', '', '', '', '', '8,219.29', 'FWT', '', '', '', '', '', '117.23', '', '', '1,288.89', '', '', '', '', '', '', '', '', '', 'ER SS', '', '', '', '', '46.45', '', '509.60']
['', 'Regular', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'SS W/H', '', '', '', '', '', '46.45', '', '', '509.60', '', '', '', '', '', '', '', '', '', 'ER MC', '', '', '', '', '10.86', '', '119.18']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'MC W/H', '', '', '', '

In [58]:
employee_blocks

[[['',
   'Emp# 137',
   '',
   '',
   '',
   '',
   'Dawn M DeRose',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   ''],
  ['',
   'DirDep',
   '',
   '',
   '',
   '',
   '',
   '',
   'Net Pay:',
   '',
   '554.07',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   ''],
  ['',
   '',
   '',
   '',
   '',
   '',
   '',
   '',
   '10.00',
   '',
   '74.9192',
   '',
   '',
   '',
   '749.19',
   '',
   '',
   '',
   '',
   '8,219.29',
   'FWT',
   '',
   '',
   '',
   '',
   '',
   '117.23',
   '',
   '',
   '1,288.89',
   '',
   '',
   '',
   '',
   '',
   '',
   '

In [62]:
file_path = "payroll register report 5325 to 51625.xlsx"
wb = load_workbook(file_path, data_only=True)
sheet = wb.active

# === 2. Extract relevant data as raw text ===
lines = []
for row in sheet.iter_rows(values_only=True):
    row_str = "\t".join(str(cell) if cell is not None else "" for cell in row)
    lines.append(row_str.strip())

raw_text = "\n".join([line for line in lines if line.strip()])
# limit prompt size
raw_text

"Payroll Register Report\nTown of New Baltimore\nPay Period From 05/03/2025 to 05/16/2025, Pay Date: 05/23/2025, Payroll # 13 (Standard)\nEarnings\t\t\t\t\t\t\t\tHours*\t\tRate\t\t\t\tCurrent\t\t\t\tYTD\t\tTaxes\t\t\t\t\t\tCurrent\t\tYTD\t\tDeductions\t\t\t\tCurrent\t\t\tYTD\t\tER Taxes**\t\t\t\t\tCurrent\t\tYTD\nDepartment :\t\t\t\tGeneral\nEmp# 137\t\t\t\t\tDawn M DeRose\nDirDep\t\t\t\t\t\t\tNet Pay:\t\t554.07\n10.00\t\t74.9192\t\t\t\t749.19\t\t\t\t\t8,219.29\tFWT\t\t\t\t\t\t117.23\t\t\t1,288.89\t\t\t\t\t\t\t\t\t\tER SS\t\t\t\t\t46.45\t\t509.60\nRegular\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSS W/H\t\t\t\t\t\t46.45\t\t\t509.60\t\t\t\t\t\t\t\t\t\tER MC\t\t\t\t\t10.86\t\t119.18\nMC W/H\t\t\t\t\t\t10.86\t\t\t119.18\nNY State Tax\t\t\t\t\t\t19.38\t\t\t212.09\nNY SDI\t\t\t\t\t\t1.20\t\t\t13.20\nEmployee Tot:\t\t\t\t\t10\t\t\t\t\t\t749.19\t\t\t\t\t8219.29\t\t\t\t\t\t\t195.12\t\t\t2142.96\t\t\t\t\t\t0\t\t\t0\t\t\t\t\t\t57.31\t\t628.78\nEmp# 160\t\t\t\t\tJessica M Diamond\nDirDep\t\t\t\t\t\t\t

In [63]:
len(raw_text)

17169

In [64]:
from openpyxl import load_workbook
import json

# Load workbook and sheet
wb = load_workbook("payroll register report 5325 to 51625.xlsx", data_only=True)
sheet = wb.active

# Step 1: Read all rows as text
rows = []
for row in sheet.iter_rows(values_only=True):
    text_row = "\t".join(str(cell).strip() if cell else "" for cell in row).strip()
    rows.append(text_row)

# Step 2: Identify start of each employee
employee_blocks = []
current_block = []
for line in rows:
    if "Emp#" in line:
        if current_block:
            employee_blocks.append(current_block)
            current_block = []
    current_block.append(line)

# Don't forget the last employee
if current_block:
    employee_blocks.append(current_block)

# Step 3: Format chunks for LLM
formatted_chunks = []
for block in employee_blocks:
    chunk = "\n".join(block)
    formatted_chunks.append(chunk)

# Step 4: Save or print a few examples
for i, chunk in enumerate(formatted_chunks[:3]):
    print(f"\n--- Employee #{i+1} Chunk ---\n{chunk}")


formatted_chunks


--- Employee #1 Chunk ---
Payroll Register Report
Town of New Baltimore
Pay Period From 05/03/2025 to 05/16/2025, Pay Date: 05/23/2025, Payroll # 13 (Standard)
Earnings								Hours*		Rate				Current				YTD		Taxes						Current		YTD		Deductions				Current			YTD		ER Taxes**					Current		YTD
Department :				General

--- Employee #2 Chunk ---
Emp# 137					Dawn M DeRose
DirDep							Net Pay:		554.07
10.00		74.9192				749.19					8,219.29	FWT						117.23			1,288.89										ER SS					46.45		509.60
Regular																			SS W/H						46.45			509.60										ER MC					10.86		119.18
MC W/H						10.86			119.18
NY State Tax						19.38			212.09
NY SDI						1.20			13.20
Employee Tot:					10						749.19					8219.29							195.12			2142.96															57.31		628.78

--- Employee #3 Chunk ---
Emp# 160					Jessica M Diamond
DirDep							Net Pay:		1099.96
65.00		20.57				1,337.05					2,704.96	FWT						82.04			135.70										ER SS					82.90		167.71
Regular																			SS W/H						82.90			167.71

['Payroll Register Report\nTown of New Baltimore\nPay Period From 05/03/2025 to 05/16/2025, Pay Date: 05/23/2025, Payroll # 13 (Standard)\nEarnings\t\t\t\t\t\t\t\tHours*\t\tRate\t\t\t\tCurrent\t\t\t\tYTD\t\tTaxes\t\t\t\t\t\tCurrent\t\tYTD\t\tDeductions\t\t\t\tCurrent\t\t\tYTD\t\tER Taxes**\t\t\t\t\tCurrent\t\tYTD\nDepartment :\t\t\t\tGeneral',
 'Emp# 137\t\t\t\t\tDawn M DeRose\nDirDep\t\t\t\t\t\t\tNet Pay:\t\t554.07\n10.00\t\t74.9192\t\t\t\t749.19\t\t\t\t\t8,219.29\tFWT\t\t\t\t\t\t117.23\t\t\t1,288.89\t\t\t\t\t\t\t\t\t\tER SS\t\t\t\t\t46.45\t\t509.60\nRegular\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSS W/H\t\t\t\t\t\t46.45\t\t\t509.60\t\t\t\t\t\t\t\t\t\tER MC\t\t\t\t\t10.86\t\t119.18\nMC W/H\t\t\t\t\t\t10.86\t\t\t119.18\nNY State Tax\t\t\t\t\t\t19.38\t\t\t212.09\nNY SDI\t\t\t\t\t\t1.20\t\t\t13.20\nEmployee Tot:\t\t\t\t\t10\t\t\t\t\t\t749.19\t\t\t\t\t8219.29\t\t\t\t\t\t\t195.12\t\t\t2142.96\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t57.31\t\t628.78',
 'Emp# 160\t\t\t\t\tJessica M Diamond\nDirDep\t\t\t\t\

In [66]:
from openpyxl import load_workbook
import json

# === Load the workbook ===
file_path = "payroll register report 5325 to 51625.xlsx"
wb = load_workbook(file_path, data_only=True)
sheet = wb.active

# === Read all rows as raw strings ===
rows = []
for row in sheet.iter_rows(values_only=True):
    raw_line = "\t".join(str(cell).strip() if cell else "" for cell in row).strip()
    rows.append(raw_line)

# === Group rows into chunks per employee (starts with 'Emp#') ===
employee_chunks = []
current_chunk = []

for line in rows:
    if "Emp#" in line:
        if current_chunk:
            employee_chunks.append("\n".join(current_chunk))
            current_chunk = []
    current_chunk.append(line)

# Add the final chunk
if current_chunk:
    employee_chunks.append("\n".join(current_chunk))


with open("employee_chunks_raw.json", "w", encoding="utf-8") as f:
    json.dump(employee_chunks, f, indent=2)

print(f"✅ Done. Extracted {len(employee_chunks)} employee chunks.")

✅ Done. Extracted 23 employee chunks.


In [89]:
with open("all_extracted_employees.json", "r") as f:
    employee_data = json.load(f)
employee_data

[{'Emp#': '137',
  'Name': 'Dawn M DeRose',
  'Department': None,
  'RegHrs': '10.00',
  'RegAmt': '74.9192',
  'RegAmt_YTD': '749.19',
  'VacHrs': None,
  'VacAmt': None,
  'VacAmt_YTD': None,
  'HolHrs': None,
  'HolAmt': None,
  'HolAmt_YTD': None,
  'SickHrs': None,
  'SickAmt': None,
  'SickAmt_YTD': None,
  'OTHrs': None,
  'OTAmt': None,
  'OTAmt_YTD': None,
  'PersonalHrs': None,
  'PersonalAmt': None,
  'PersonalAmt_YTD': None,
  'Deputy Clerk Hrs': None,
  'Deputy Clerk Amt': None,
  'Deputy Clerk Amt_YTD': None,
  'OtherHrs': None,
  'OtherAmt': None,
  'OtherAmt_YTD': None,
  'Emergency Mgmt Hrs': None,
  'Emergency Mgmt Amt': None,
  'Emergency Mgmt Amt_YTD': None,
  'FWT': '117.23',
  'FWT_YTD': '1,288.89',
  'SS W/H': '46.45',
  'SS W/H_YTD': '509.60',
  'MC W/H': '10.86',
  'MC W/H_YTD': '119.18',
  'SOCSEC': None,
  'SOCSEC_YTD': None,
  'MEDI': None,
  'MEDI_YTD': None,
  'NY State Tax': '19.38',
  'NY State Tax_YTD': '212.09',
  'NY SDI': '1.20',
  'NY SDI_YTD': '13.

In [90]:
all_keys = set()
for entry in employee_data:
    if isinstance(entry, dict):
        all_keys.update(entry.keys())
all_keys

{'414(h)',
 '414(h)_YTD',
 '457(b)',
 '457(b)_YTD',
 'Aflac',
 'Aflac Pre-Tax',
 'Aflac Pre-Tax_YTD',
 'Aflac_YTD',
 'Dental Ins',
 'Dental Ins_YTD',
 'Department',
 'Deputy Clerk Amt',
 'Deputy Clerk Amt_YTD',
 'Deputy Clerk Hrs',
 'ER MC',
 'ER MC_YTD',
 'ER SS',
 'ER SS_YTD',
 'Emergency Mgmt Amt',
 'Emergency Mgmt Amt_YTD',
 'Emergency Mgmt Hrs',
 'Emp#',
 'FWT',
 'FWT_YTD',
 'HolAmt',
 'HolAmt_YTD',
 'HolHrs',
 'Loan Repayment',
 'Loan Repayment_YTD',
 'MC W/H',
 'MC W/H_YTD',
 'MEDI',
 'MEDI_YTD',
 'Medical Ins',
 'Medical Ins_YTD',
 'NY SDI',
 'NY SDI_YTD',
 'NY State Tax',
 'NY State Tax_YTD',
 'Name',
 'Net Pay',
 'OTAmt',
 'OTAmt_YTD',
 'OTHrs',
 'OtherAmt',
 'OtherAmt_YTD',
 'OtherHrs',
 'PersonalAmt',
 'PersonalAmt_YTD',
 'PersonalHrs',
 'Pre Tax SCP',
 'Pre Tax SCP_YTD',
 'RegAmt',
 'RegAmt_YTD',
 'RegHrs',
 'SOCSEC',
 'SOCSEC_YTD',
 'SS W/H',
 'SS W/H_YTD',
 'SickAmt',
 'SickAmt_YTD',
 'SickHrs',
 'Total Deductions YTD',
 'Total ER Taxes Cuurrent',
 'Total ER Taxes YTD',


In [91]:
column_to_json_key = {
    # Standard earnings & hours
    "RegHrs": "RegHrs",
    "RegAmt": "RegAmt",
    "VacHrs": "VacHrs",
    "VacAmt": "VacAmt",
    "HolHrs": "HolHrs",
    "HolAmt": "HolAmt",
    "OTHrs": "OTHrs",
    "OTAmt": "OTAmt",
    
    # Tax fields
    "FEDTAX": "FWT",
    "SOCSEC": "SS W/H",
    "MEDI": "MC W/H",
    "NYSTATE": "NY State Tax",
    "NYSDI": "NY SDI",

    # Leave and personal hours
    "PersonalHrs": "PersonalHrs",
    "PersonalAmt": "PersonalAmt",
    "SickHrs": "SickHrs",
    "SickAmt": "SickAmt",
    "DeputHrs": "Deputy Clerk Hrs",
    "DeputAmt": "Deputy Clerk Amt",

    # Special compensation
    "E mgtAmt": "Emergency Mgmt Amt",
    
    # Deductions (by code)
    "DED514791": "414(h)",
    "DED514789": "457(b)",
    "DED514786": "Aflac",
    "DED514796": "Aflac Pre-Tax",
    "DED514797": "Dental Ins",
    "DED514795": "Loan Repayment",
    "DED514800": "Medical Ins",
    "DED514792": "Pre Tax SCP",
    "DED514802": "Union Dues",
    "DED514799": "Vision Ins",

    # Net pay
    "NetAmt": "Net Pay"
}


In [92]:
import csv
import json

# Load JSON data
json_path = "all_extracted_employees.json"
csv_input_path = "NewBaltimo 532025 to 5162025.csv"
csv_output_path = "NewBaltimo_Filled.csv"

with open(json_path, "r") as f:
    employee_data = json.load(f)
emp_lookup = {emp["Emp#"]: emp for emp in employee_data}

# Define mapping from CSV columns to JSON keys
column_to_json_key = {
    # Standard earnings & hours
    "RegHrs": "RegHrs",
    "RegAmt": "RegAmt",
    "VacHrs": "VacHrs",
    "VacAmt": "VacAmt",
    "HolHrs": "HolHrs",
    "HolAmt": "HolAmt",
    "OTHrs": "OTHrs",
    "OTAmt": "OTAmt",

    # Tax fields
    "FEDTAX": "FWT",
    "SOCSEC": "SS W/H",
    "MEDI": "MC W/H",
    "NYSTATE": "NY State Tax",
    "NYSDI": "NY SDI",

    # Leave and personal hours
    "PersonalHrs": "PersonalHrs",
    "PersonalAmt": "PersonalAmt",
    "SickHrs": "SickHrs",
    "SickAmt": "SickAmt",
    "DeputHrs": "Deputy Clerk Hrs",
    "DeputAmt": "Deputy Clerk Amt",

    # Special compensation
    "E mgtAmt": "Emergency Mgmt Amt",

    # Deductions (by code)
    "DED514791": "414(h)",
    "DED514789": "457(b)",
    "DED514786": "Aflac",
    "DED514796": "Aflac Pre-Tax",
    "DED514797": "Dental Ins",
    "DED514795": "Loan Repayment",
    "DED514800": "Medical Ins",
    "DED514792": "Pre Tax SCP",
    "DED514802": "Union Dues",
    "DED514799": "Vision Ins",

    # Net pay
    "NetAmt": "Net Pay"
}

# Read and populate the CSV
with open(csv_input_path, "r", newline='', encoding="utf-8") as infile, \
     open(csv_output_path, "w", newline='', encoding="utf-8") as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    headers = []
    current_emp_id = None
    current_emp_data = None

    for row in reader:
        if not headers:
            headers = row
            writer.writerow(row)
            continue

        # Identify Emp# row
        if row and isinstance(row[0], str) and row[0].strip().startswith("Emp#"):
            parts = row[0].split()
            if len(parts) > 1:
                current_emp_id = parts[1].strip()
                current_emp_data = emp_lookup.get(current_emp_id)

        if current_emp_data:
            for i, col in enumerate(headers):
                json_key = column_to_json_key.get(col)
                if json_key and json_key in current_emp_data:
                    row[i] = str(current_emp_data[json_key]) if current_emp_data[json_key] is not None else ""

        writer.writerow(row)

csv_output_path  # Return path to filled CSV


'NewBaltimo_Filled.csv'

In [95]:
template_path = "NewBaltimo 532025 to 5162025.csv"

# Step 1: Read only the header row (row 8, zero-indexed = 7)
full_df = pd.read_csv(template_path, header=None, skip_blank_lines=False)
header_row = full_df.iloc[7].fillna("").astype(str).tolist()
header_row

['',
 '',
 'S/H',
 'DEPT',
 'RegHrs',
 'RegAmt',
 'VacHrs',
 'VacAmt',
 'HolHrs',
 'HolAmt',
 'ReimbAmt',
 'OTHrs',
 'OTAmt',
 'SickHrs',
 'SickAmt',
 'PersHrs',
 'PersAmt',
 'DeputHrs',
 'DeputAmt',
 'RecorHrs',
 'RecorAmt',
 'CompHrs',
 'CompAmt',
 'ClerkHrs',
 'ClerkAmt',
 'JuryHrs',
 'JuryAmt',
 'E MgtAmt',
 'BRVHrs',
 'BRVAmt',
 'RetroAmt',
 'DepStAmt',
 'SupvSAmt',
 'AssesHrs',
 'AssesAmt',
 'CodesHrs',
 'CodesAmt',
 'ZoninHrs',
 'ZoninAmt',
 'PlannHrs',
 'PlannAmt',
 'TaxHrs',
 'TaxAmt',
 'FEDTAX',
 'SOCSEC',
 'MEDI',
 'NYSTATE',
 'NYPFML',
 'NYSDI',
 'DED514790',
 'DED514791',
 'DED514789',
 'DED521466',
 'DED514786',
 'DED514796',
 'DED514793',
 'DED514787',
 'DED514788',
 'DED514797',
 'DED514794',
 'DED514795',
 'DED514800',
 'DED514798',
 'DED514792',
 'DED514801',
 'DED514802',
 'DED514799',
 'DED514803',
 'NetAmt']

In [103]:
column_to_json_key = {
    # Identity
    "Emp Num": "Emp#",
    "Employee Name": "Name",

    # Regular pay
    "Regular Hours": "RegHrs",
    "Regular Amount": "RegAmt",

    # Vacation
    "Vacation Hours": "VacHrs",
    "Vacation Amount": "VacAmt",

    # Holiday
    "Holiday Hours": "HolHrs",
    "Holiday Amount": "HolAmt",

    # Overtime
    "Overtime Hours": "OTHrs",
    "Overtime Amount": "OTAmt",

    # Sick
    "Sick Hours": "SickHrs",
    "Sick Amount": "SickAmt",

    # Personal
    "Personal Hours": "PersonalHrs",
    "Personal Amount": "PersonalAmt",

    # Deputy Clerk
    "Deputy Clerk 1410 Hours": "Deputy Clerk Hrs",
    "Deputy Clerk 1410 Amount": "Deputy Clerk Amt",

    # Emergency Mgmt
    "Emergency Mgmt Amount": "Emergency Mgmt Amt",

    # Taxes
    "Federal Tax": "FWT",
    "Soc.Sec. Tax": "SS W/H",
    "Medicare Tax": "MC W/H",
    "NY State Tax": "NY State Tax",
    "NY SDI Tax": "NY SDI",

    # Deductions
    "414(H) Amount": "414(h)",
    "457(b) Amount": "457(b)",
    "Aflac Amount": "Aflac",
    "Aflac Pre-Tax Amount": "Aflac Pre-Tax",
    "Dental Ins Amount": "Dental Ins",
    "Loan Repayment Amount": "Loan Repayment",
    "Medical Ins Amount": "Medical Ins",
    "Pre Tax SCP Amount": "Pre Tax SCP",
    "Union Dues Amount": "Union Dues",
    "Vision Ins Amount": "Vision Ins",

    # Net
    "Net Amount": "Net Pay"
}


In [108]:
import csv
import json

# === File Paths ===
csv_path = "NewBaltimo 532025 to 5162025.csv"
json_path = "all_extracted_employees.json"
output_csv = "populated_output.csv"

# === Load JSON data ===
with open(json_path, "r", encoding="utf-8") as f:
    json_data = json.load(f)
json_map = {emp["Emp#"]: emp for emp in json_data}

# === Load CSV as list of rows ===
with open(csv_path, newline='', encoding='utf-8') as f:
    reader = list(csv.reader(f))

# === Identify header rows (row 9 and 10 are index 8 and 9) ===
header_row_1 = reader[8]
header_row_2 = reader[9]

# === Merge headers to get final column names ===
final_headers = []
for h1, h2 in zip(header_row_1, header_row_2):
    merged = f"{h1.strip()} {h2.strip()}".strip()
    final_headers.append(merged)

# === Mapping: Human-readable to JSON key ===
column_to_json_key = {
    "Emp Num": "Emp#",
    "Employee Name": "Name",
    "Regular Hours": "RegHrs",
    "Regular Amount": "RegAmt",
    "Vacation Hours": "VacHrs",
    "Vacation Amount": "VacAmt",
    "Holiday Hours": "HolHrs",
    "Holiday Amount": "HolAmt",
    "Overtime Hours": "OTHrs",
    "Overtime Amount": "OTAmt",
    "Sick Hours": "SickHrs",
    "Sick Amount": "SickAmt",
    "Personal Hours": "PersonalHrs",
    "Personal Amount": "PersonalAmt",
    "Deputy Clerk 1410 Hours": "Deputy Clerk Hrs",
    "Deputy Clerk 1410 Amount": "Deputy Clerk Amt",
    "Emergency Mgmt Amount": "Emergency Mgmt Amt",
    "Federal Tax": "FWT",
    "Soc.Sec. Tax": "SS W/H",
    "Medicare Tax": "MC W/H",
    "NY State Tax": "NY State Tax",
    "NY SDI Tax": "NY SDI",
    "414(H) Amount": "414(h)",
    "457(b) Amount": "457(b)",
    "Aflac Amount": "Aflac",
    "Aflac Pre-Tax Amount": "Aflac Pre-Tax",
    "Dental Ins Amount": "Dental Ins",
    "Loan Repayment Amount": "Loan Repayment",
    "Medical Ins Amount": "Medical Ins",
    "Pre Tax SCP Amount": "Pre Tax SCP",
    "Union Dues Amount": "Union Dues",
    "Vision Ins Amount": "Vision Ins",
    "Net Amount": "Net Pay"
}

# === Build column index map ===
header_index_map = {col: i for i, col in enumerate(final_headers)}

# === Process and update rows starting from row 10 (index 9) onward ===
for i in range(10, len(reader)):
    row = reader[i]
    emp_num_raw = row[0].strip()
    if emp_num_raw.isdigit():
        emp_num = emp_num_raw
        if emp_num in json_map:
            emp_json = json_map[emp_num]
            for col_name, json_key in column_to_json_key.items():
                col_idx = header_index_map.get(col_name)
                if col_idx is not None and json_key in emp_json:
                    val = emp_json[json_key]
                    if val is not None:
                        row[col_idx] = str(val)

# === Write updated CSV ===
with open(output_csv, "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(reader)

print(f"✅ Done. Populated data saved to: {output_csv}")


✅ Done. Populated data saved to: populated_output.csv


In [105]:
template_path = "NewBaltimo 532025 to 5162025.csv"
json_path = "all_extracted_employees.json"

# Step 1: Read the full CSV
full_df = pd.read_csv(template_path, header=None, skip_blank_lines=False)

# Step 2: Extract and combine header rows (row 8 and 9 → index 7 and 8)
row_8 = full_df.iloc[7].fillna("").astype(str).tolist()
row_9 = full_df.iloc[8].fillna("").astype(str).tolist()
combined_header = [(h1 + " " + h2).strip() if (h1 + " " + h2).strip() else h1 or h2 for h1, h2 in zip(row_8, row_9)]

# Step 3: Read the employee table using the combined header
data_df = pd.read_csv(template_path, skiprows=9, names=combined_header)

# Step 4: Rename identity columns for easier matching
data_df.columns.values[0:2] = ["Emp#", "Name"]

# Step 5: Load employee JSON data and create lookup
with open(json_path, "r") as f:
    employee_data = json.load(f)
emp_lookup = {emp["Emp#"]: emp for emp in employee_data}

for idx, row in data_df.iterrows():
    emp_id = str(row["Emp#"]).strip()
    if emp_id in emp_lookup:
        emp_data = emp_lookup[emp_id]
        for col, json_key in column_to_json_key.items():
            if col in data_df.columns and json_key in emp_data:
                data_df.at[idx, col] = emp_data[json_key]
    # If employee not found in JSON, row is untouched

# Step 8: Combine top rows + populated table
top_rows = full_df.iloc[:7]
header_row_df = pd.DataFrame([combined_header], columns=top_rows.columns)
final_df = pd.concat([top_rows, header_row_df, data_df], ignore_index=True)

# Step 9 (resave): Save the partially populated CSV (only for known employees)
partial_populated_path = "NewBaltimo_PartiallyPopulated.csv"
final_df.to_csv(partial_populated_path, index=False, header=False)

partial_populated_path

'NewBaltimo_PartiallyPopulated.csv'

In [102]:
column_to_json_key = {
    # Identity
    "Emp Num": "Emp#",
    "Employee Name": "Name",

    # Regular pay
    "Regular Hours": "RegHrs",
    "Regular Amount": "RegAmt",

    # Vacation
    "Vacation Hours": "VacHrs",
    "Vacation Amount": "VacAmt",

    # Holiday
    "Holiday Hours": "HolHrs",
    "Holiday Amount": "HolAmt",

    # Overtime
    "Overtime Hours": "OTHrs",
    "Overtime Amount": "OTAmt",

    # Sick
    "Sick Hours": "SickHrs",
    "Sick Amount": "SickAmt",

    # Personal
    "Personal Hours": "PersonalHrs",
    "Personal Amount": "PersonalAmt",

    # Deputy Clerk
    "Deputy Clerk 1410 Hours": "Deputy Clerk Hrs",
    "Deputy Clerk 1410 Amount": "Deputy Clerk Amt",

    # Emergency Mgmt
    "Emergency Mgmt Amount": "Emergency Mgmt Amt",

    # Taxes
    "Federal Tax": "FWT",
    "Soc.Sec. Tax": "SS W/H",
    "Medicare Tax": "MC W/H",
    "NY State Tax": "NY State Tax",
    "NY SDI Tax": "NY SDI",

    # Deductions
    "414(H) Amount": "414(h)",
    "457(b) Amount": "457(b)",
    "Aflac Amount": "Aflac",
    "Aflac Pre-Tax Amount": "Aflac Pre-Tax",
    "Dental Ins Amount": "Dental Ins",
    "Loan Repayment Amount": "Loan Repayment",
    "Medical Ins Amount": "Medical Ins",
    "Pre Tax SCP Amount": "Pre Tax SCP",
    "Union Dues Amount": "Union Dues",
    "Vision Ins Amount": "Vision Ins",

    # Net
    "Net Amount": "Net Pay"
}


In [97]:
import pandas as pd

# Load the original CSV, skipping the irrelevant rows and parsing headers


# Step 2: Deduplicate headers before applying them
deduped_header = []
seen = {}
for col in header_row:
    if col not in seen:
        seen[col] = 1
        deduped_header.append(col)
    else:
        seen[col] += 1
        deduped_header.append(f"{col}_{seen[col]}")

# Now load the data using deduped headers
#data_df = pd.read_csv(template_path, skiprows=9, names=deduped_header)

# Step 2: Create the DataFrame using header and skipping the first 9 rows
data_df = pd.read_csv(template_path, skiprows=9, names=deduped_header)

# Step 3: Rename first two columns as "Emp#", "Name" explicitly
data_df.columns.values[0:2] = ["Emp#", "Name"]

# Step 4: Load employee JSON data
with open(json_path, "r") as f:
    employee_data = json.load(f)
emp_lookup = {emp["Emp#"]: emp for emp in employee_data}

# Step 5: Define the mapping again (reuse previous mapping)
column_to_json_key = {
    "RegHrs": "RegHrs",
    "RegAmt": "RegAmt",
    "VacHrs": "VacHrs",
    "VacAmt": "VacAmt",
    "HolHrs": "HolHrs",
    "HolAmt": "HolAmt",
    "OTHrs": "OTHrs",
    "OTAmt": "OTAmt",
    "FEDTAX": "FWT",
    "SOCSEC": "SS W/H",
    "MEDI": "MC W/H",
    "NYSTATE": "NY State Tax",
    "NYSDI": "NY SDI",
    "PersonalHrs": "PersonalHrs",
    "PersonalAmt": "PersonalAmt",
    "SickHrs": "SickHrs",
    "SickAmt": "SickAmt",
    "DeputHrs": "Deputy Clerk Hrs",
    "DeputAmt": "Deputy Clerk Amt",
    "E mgtAmt": "Emergency Mgmt Amt",
    "DED514791": "414(h)",
    "DED514789": "457(b)",
    "DED514786": "Aflac",
    "DED514796": "Aflac Pre-Tax",
    "DED514797": "Dental Ins",
    "DED514795": "Loan Repayment",
    "DED514800": "Medical Ins",
    "DED514792": "Pre Tax SCP",
    "DED514802": "Union Dues",
    "DED514799": "Vision Ins",
    "NetAmt": "Net Pay"
}

# Step 6: Populate data using mapping
for idx, row in data_df.iterrows():
    emp_id = str(row["Emp#"]).strip()
    if emp_id in emp_lookup:
        emp_data = emp_lookup[emp_id]
        for col, json_key in column_to_json_key.items():
            if col in data_df.columns and json_key in emp_data:
                data_df.at[idx, col] = emp_data[json_key]

# Step 7: Concatenate top 9 rows + updated table
top_rows = full_df.iloc[:9]
updated_table = pd.concat([top_rows, pd.DataFrame([data_df.columns.tolist()], columns=top_rows.columns), data_df], ignore_index=True)

# Step 8: Save to a new filled CSV
final_filled_path = "NewBaltimo_Filled_Formatted.csv"
updated_table.to_csv(final_filled_path, index=False, header=False)

final_filled_path


'NewBaltimo_Filled_Formatted.csv'

In [78]:
with open(aw_temp_path, newline='', encoding='utf-8') as infile:
    
    reader = csv.reader(infile)
    header_row = None
    for row in reader:
        print(row)


['\ufeffFirm:', '', 'Evening Star Bookkeeping Service Inc', '', '', 'Firm Code:', '', 'STARB3966', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Employer:', '', 'Town of New Baltimore', '', '', 'Client Code:', '', 'NewBaltimo', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Pay Period:', '', '5/3/2025 to 5/16/2025', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Pay Schedule:', '

In [87]:
import csv
import json

# Load JSON data
with open("all_extracted_employees.json", "r") as f:
    employees = json.load(f)
emp_lookup = {emp["Emp#"]: emp for emp in employees}

# Load the original template
input_csv = "NewBaltimo 532025 to 5162025.csv"
output_csv = "NewBaltimo_Filled.csv"

with open(input_csv, "r", newline='', encoding="utf-8") as infile, \
     open(output_csv, "w", newline='', encoding="utf-8") as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    for row in reader:
        updated_row = row.copy()
        # Find row with Emp# in first column like "Emp# 137"
        if row and isinstance(row[0], str) and row[0].strip().startswith("Emp#"):
            emp_id = row[0].split("#")[1].strip()
            employee_data = emp_lookup.get(emp_id)

            if employee_data:
                # Look ahead to update Net Pay, RegHrs, RegAmt, etc.
                for i, cell in enumerate(row):
                    if "Net Pay" in cell:
                        updated_row[i+1] = str(employee_data.get("NetAmt", ""))
            # Optional: You can update more fields here if you know their exact column position

        writer.writerow(updated_row)


In [26]:
aw_template_path = "TownSharon 6-30-2025 to 6-30-2025.csv"
employee_master_path = "TownSharon Employees (4).csv"
pdf_path = "2025-01-03 1 Payroll Set.pdf"
output_csv_path = "TownSharon_Processed_01032025.csv"

In [27]:

aw_template_df = pd.read_csv(aw_template_path,aw_temp_new_baltimore_path)  # skip first 10 header rows
employee_master_df = pd.read_csv(employee_master_path)

TypeError: read_csv() takes 1 positional argument but 2 were given

In [4]:
aw_template_df

,235,Barbic III William,S,General,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,0.0000.4,0.0000.5,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,0.0000.6,0.0000.7
0,196.0,Barlow William J.,H,General,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,222.0,Booth Andrew J.,H,Highway,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,266.0,Brown Rosa M.,H,Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,241.0,Burton Ava R.,H,Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,223.0,Countryman Robert L.,H,Cemetery,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,226.0,Weller Hanna L.,H,Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
69,219.0,Wilday Erin T.,H,Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
70,246.0,Wilkinson Alyssa B.,H,Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
71,124.0,Young Brian,H,General,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:


aw_template_df = pd.read_csv(aw_template_path, skiprows=10)  # skip first 10 header rows
employee_master_df = pd.read_csv(employee_master_path)

# Read and extract payroll data from PDF
def extract_payroll_data(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()

    # Define structure to collect employee data
    employee_data = []

    # Use regex to extract each employee block
    employee_blocks = re.split(r"Emp#\s+", text)[1:]  # split and skip preamble
    for block in employee_blocks:
        try:
            emp_id_search = re.match(r"(\d+)\s+([A-Za-z ,.'\-]+)\s+Dept\s*:\s*([A-Za-z ]+)", block)
            if not emp_id_search:
                continue
            emp_id, name, dept = emp_id_search.groups()
            emp_id = int(emp_id.strip())
            name = name.strip()
            dept = dept.strip()

            # Extract hours and regular pay
            reg_match = re.search(r"Regular\s+([\d.]+)\s+[\d.]+\s+([\d,]+\.\d+)", block)
            reg_hrs = float(reg_match.group(1)) if reg_match else 0.0
            reg_amt = float(reg_match.group(2).replace(",", "")) if reg_match else 0.0

            # Taxes
            fwt = re.search(r"FWT\s+([\d.]+)", block)
            ss = re.search(r"SS W/H\s+([\d.]+)", block)
            mc = re.search(r"MC W/H\s+([\d.]+)", block)
            nystate = re.search(r"NY State Tax\s+([\d.]+)", block)

            fedtax = float(fwt.group(1)) if fwt else 0.0
            socsec = float(ss.group(1)) if ss else 0.0
            medi = float(mc.group(1)) if mc else 0.0
            nystate_tax = float(nystate.group(1)) if nystate else 0.0

            # Deductions (like 414(H))
            ded414h = re.search(r"414\(H\)\s+([\d.]+)", block)
            ded_414h = float(ded414h.group(1)) if ded414h else 0.0

            employee_data.append({
                "Emp": emp_id,
                "Name": name,
                "Dept": dept,
                "RegHrs": reg_hrs,
                "RegAmt": reg_amt,
                "FEDTAX": fedtax,
                "SOCSEC": socsec,
                "MEDI": medi,
                "NYSTATE": nystate_tax,
                "DED514791": ded_414h
            })
        except Exception as e:
            print(f"Error parsing block:\n{block}\nError: {e}")
            continue

    return pd.DataFrame(employee_data)

# Extract data from PDF
payroll_df = extract_payroll_data(pdf_path)

# Merge into AW template
merged_df = aw_template_df.copy()
for idx, row in payroll_df.iterrows():
    emp_id = row["Emp"]
    match = merged_df["Emp"].astype(str) == str(emp_id)
    if match.any():
        merged_df.loc[match, "RegHrs"] = row["RegHrs"]
        merged_df.loc[match, "RegAmt"] = row["RegAmt"]
        merged_df.loc[match, "FEDTAX"] = row["FEDTAX"]
        merged_df.loc[match, "SOCSEC"] = row["SOCSEC"]
        merged_df.loc[match, "MEDI"] = row["MEDI"]
        merged_df.loc[match, "NYSTATE"] = row["NYSTATE"]
        merged_df.loc[match, "DED514791"] = row["DED514791"]

# Save result to new CSV
header_rows = []
with open(aw_template_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for _ in range(10):
        header_rows.append(next(reader))

with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(header_rows)
    merged_df.to_csv(f, index=False)

output_csv_path  # Return path to download the new file


In [2]:
import fitz  # PyMuPDF
import re
import json
import os
BASE_DIR="Data"
INPUT_DIR = os.path.join(BASE_DIR, "input_files")
file_name = "2025-01-03 1 Payroll Set.pdf"

pdf_path =  os.path.join(INPUT_DIR, file_name)
doc = fitz.open(pdf_path)

text = ""
for page in doc:
    text += page.get_text()
    
text

"General\nDepartment :\nEmp# 107\nDavid R Cross\nDept : General\nDirDep\nRegular\n8.00\n18.75\n150.00\n150.00\n138.52\n0.00\n9.30\n2.18\nFWT\nSS W/H\nMC W/H\nNet Pay:\nER SS\nER MC\n9.30\n2.18\n9.30\n2.18\n0.00\n9.30\n2.18\nEmployee Tot:\n8.00\n150.00\n150.00\n11.48\n0.00\n11.48\n11.48\n0.00\n11.48\nEmp# 231\nLorraine A Gunderson\nDept : General\nDirDep\nRegular\n31.50\n19.00\n598.50\n598.50\n402.71\n150.00\n37.11\n8.68\nFWT\nSS W/H\nMC W/H\nNet Pay:\nER SS\nER MC\n37.11\n8.68\n37.11\n8.68\n150.00\n37.11\n8.68\nEmployee Tot:\n31.50\n598.50\n598.50\n195.79\n0.00\n45.79\n45.79\n0.00\n195.79\nEmp# 169\nEllen J Hall\nDept : General\nDirDep\nRegular\n24.00\n21.5278\n516.67\n516.67\n477.15\n0.00\n32.03\n7.49\nFWT\nSS W/H\nMC W/H\nNet Pay:\nER SS\nER MC\n32.03\n7.49\n32.03\n7.49\n0.00\n32.03\n7.49\nEmployee Tot:\n24.00\n516.67\n516.67\n39.52\n0.00\n39.52\n39.52\n0.00\n39.52\nEmp# 268\nKenneth J Kelly III\nNew Hire\nDept : General\nDirDep\nRegular\n40.00\n26.0417\n1,041.67\n1,041.67\n946.82\n0

In [121]:
import fitz  # PyMuPDF
import os

BASE_DIR = "Data"
INPUT_DIR = os.path.join(BASE_DIR, "input_files")
file_name = "2025-01-03 1 Payroll Set.pdf"

pdf_path = os.path.join(INPUT_DIR, file_name)
doc = fitz.open(pdf_path)

for i, page in enumerate(doc):
    print(f"\n===== Page {i + 1} Blocks =====\n")
    blocks = page.get_text("blocks")
    for block in blocks:
        print(block)



===== Page 1 Blocks =====

(26.27199935913086, 97.8900146484375, 130.91998291015625, 111.67998504638672, 'General\nDepartment :\n', 0, 0)
(26.27199935913086, 118.12944793701172, 479.7385559082031, 129.4949493408203, 'Emp# 107\nDavid R Cross\nDept : General\n', 1, 0)
(26.27199935913086, 132.439453125, 51.93775177001953, 143.7749481201172, 'DirDep\n', 2, 0)
(26.27199935913086, 147.13616943359375, 278.50286865234375, 158.5016632080078, 'Regular\n8.00\n18.75\n150.00\n150.00\n', 3, 0)
(164.15199279785156, 132.40945434570312, 189.3722686767578, 143.7449493408203, '138.52\n', 4, 0)
(387.9070129394531, 147.13616943359375, 403.9532775878906, 177.44505310058594, '0.00\n9.30\n2.18\n', 5, 0)
(283.6000061035156, 147.13616943359375, 314.73553466796875, 177.44505310058594, 'FWT\nSS W/H\nMC W/H\n', 6, 0)
(127.79199981689453, 132.40945434570312, 159.40602111816406, 143.7449493408203, 'Net Pay:\n', 7, 0)
(627.760009765625, 147.13616943359375, 654.3250122070312, 167.95835876464844, 'ER SS\nER MC\n', 8, 

In [124]:
for i, page in enumerate(doc):
    print(f"\n===== Page {i + 1} Structured Layout =====\n")
    layout = page.get_text("html")
    #rint(json.dumps(layout, indent=2))  # Pretty print
    print(layout)



===== Page 1 Structured Layout =====

<div id="page0" style="width:796.0pt;height:612.0pt">
<p style="top:100.6pt;left:93.7pt;line-height:10.0pt"><b><span style="font-family:Arial,sans-serif;font-size:10.0pt">General</span></b></p>
<p style="top:100.6pt;left:26.3pt;line-height:10.0pt"><span style="font-family:Arial,sans-serif;font-size:10.0pt">Department :</span></p>
<p style="top:120.4pt;left:26.3pt;line-height:8.2pt"><span style="font-family:Arial,sans-serif;font-size:8.2pt">Emp# 107</span></p>
<p style="top:120.4pt;left:108.0pt;line-height:8.2pt"><span style="font-family:Arial,sans-serif;font-size:8.2pt">David R Cross</span></p>
<p style="top:120.4pt;left:426.1pt;line-height:8.2pt"><span style="font-family:Arial,sans-serif;font-size:8.2pt">Dept : General</span></p>
<p style="top:134.7pt;left:26.3pt;line-height:8.2pt"><span style="font-family:Arial,sans-serif;font-size:8.2pt">DirDep</span></p>
<p style="top:149.4pt;left:26.3pt;line-height:8.2pt"><span style="font-family:Arial,sans-s

In [128]:
import pdfplumber

with pdfplumber.open("Data/input_files/2025-01-03 1 Payroll Set.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                print(row)


In [129]:
with pdfplumber.open("Data/input_files/2025-01-03 1 Payroll Set.pdf") as pdf:
    print(pdf)

In [132]:
pip install tabula-py

     --------------------------------------- 12.0/12.0 MB 46.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tabula
import os
# This reads all tables from the first page into a list of DataFrames
# 'lattice=True' is good for tables with clear grid lines
# 'stream=True' is for tables defined by whitespace
BASE_DIR="Data"
INPUT_DIR = os.path.join(BASE_DIR, "input_files")
file_name = "2025-01-03 1 Payroll Set.pdf"

pdf_path =  os.path.join(INPUT_DIR, file_name)
dfs = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

# Now dfs is a list of pandas DataFrames
dfs[0]

,arnings,Hours* Rate,Unnamed: 0,Current,YTD Taxes,Unnamed: 1,Current.1,YTD Deductions,Unnamed: 2,Current.2,YTD,ER Taxes**,Current.3,YTD.1
0,Department :,General,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Emp# 107,David R Cross,NaN,NaN,NaN,NaN,NaN,Dept : General,NaN,NaN,NaN,NaN,NaN,NaN
2,DirDep,Net Pay: 138.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Regular,8.00 18.75,NaN,150.00,150.00 FWT,NaN,0.00,0.00,NaN,NaN,NaN,ER SS,9.30,9.30
4,NaN,NaN,NaN,NaN,SS W/H,NaN,9.30,9.30,NaN,NaN,NaN,ER MC,2.18,2.18
5,NaN,NaN,NaN,NaN,MC W/H,NaN,2.18,2.18,NaN,NaN,NaN,NaN,NaN,NaN
6,Employee Tot:,8.00,NaN,150.00,150.00,NaN,11.48,11.48,NaN,0.0,0.0,NaN,11.48,11.48
7,Emp# 231,Lorraine A Gunderson,NaN,NaN,NaN,NaN,NaN,Dept : General,NaN,NaN,NaN,NaN,NaN,NaN
8,DirDep,Net Pay: 402.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Regular,31.50 19.00,NaN,598.50,598.50 FWT,NaN,150.00,150.00,NaN,NaN,NaN,ER SS,37.11,37.11


In [9]:
import pandas as pd

# Flatten DataFrame into list of text lines
lines = dfs[0].astype(str).apply(lambda row: ' '.join(row.dropna()), axis=1).tolist()

# Group lines by employee (starts with 'Emp#')
employee_blocks = []
current_block = []
for line in lines:
    if line.startswith("Emp#"):
        if current_block:
            employee_blocks.append("\n".join(current_block))
            current_block = []
    current_block.append(line)
# Add the last block
if current_block:
    employee_blocks.append("\n".join(current_block))



['Department : General nan nan nan nan nan nan nan nan nan nan nan nan',
 'Emp# 107 David R Cross nan nan nan nan nan Dept : General nan nan nan nan nan nan\nDirDep Net Pay: 138.52 nan nan nan nan nan nan nan nan nan nan nan nan\nRegular 8.00 18.75 nan 150.00 150.00 FWT nan 0.0 0.00 nan nan nan ER SS 9.3 9.3\nnan nan nan nan SS W/H nan 9.3 9.30 nan nan nan ER MC 2.18 2.18\nnan nan nan nan MC W/H nan 2.18 2.18 nan nan nan nan nan nan\nEmployee Tot: 8.00 nan 150.00 150.00 nan 11.48 11.48 nan 0.0 0.0 nan 11.48 11.48',
 'Emp# 231 Lorraine A Gunderson nan nan nan nan nan Dept : General nan nan nan nan nan nan\nDirDep Net Pay: 402.71 nan nan nan nan nan nan nan nan nan nan nan nan\nRegular 31.50 19.00 nan 598.50 598.50 FWT nan 150.0 150.00 nan nan nan ER SS 37.11 37.11\nnan nan nan nan SS W/H nan 37.11 37.11 nan nan nan ER MC 8.68 8.68\nnan nan nan nan MC W/H nan 8.68 8.68 nan nan nan nan nan nan\nEmployee Tot: 31.50 nan 598.50 598.50 nan 195.79 195.79 nan 0.0 0.0 nan 45.79 45.79',
 'Emp# 16

In [10]:
import re

def parse_block(block):
    emp_data = {}
    
    # Emp# and Name
    emp_match = re.search(r"Emp#\s+(\d+)\s+(.*)", block)
    if emp_match:
        emp_data["Emp#"] = emp_match.group(1).strip()
        emp_data["Name"] = emp_match.group(2).strip()
    
    # Net Pay
    net_pay_match = re.search(r"Net Pay:\s*([\d.,]+)", block)
    if net_pay_match:
        emp_data["Net Pay"] = float(net_pay_match.group(1).replace(",", ""))
    
    # Regular Hours and Rate
    reg_match = re.search(r"Regular\s+([\d.]+)\s+([\d.]+)", block)
    if reg_match:
        emp_data["Regular Hours"] = float(reg_match.group(1))
        emp_data["Rate"] = float(reg_match.group(2))
    
    # Regular Pay
    reg_pay_match = re.search(r"Regular.*?([\d.,]+)\s+[\d.,]+\s+FWT", block)
    if reg_pay_match:
        emp_data["Regular Pay"] = float(reg_pay_match.group(1).replace(",", ""))
    
    # FWT, SS W/H, MC W/H
    tax_matches = re.findall(r"(FWT|SS W/H|MC W/H)\s+([\d.]+)", block)
    for label, value in tax_matches:
        emp_data[label] = float(value)
    
    # ER SS and ER MC
    er_ss_match = re.search(r"ER SS\s+([\d.]+)", block)
    er_mc_match = re.search(r"ER MC\s+([\d.]+)", block)
    if er_ss_match:
        emp_data["ER SS"] = float(er_ss_match.group(1))
    if er_mc_match:
        emp_data["ER MC"] = float(er_mc_match.group(1))

    return emp_data

# Parse all employee blocks
structured_data = [parse_block(block) for block in employee_blocks]


In [12]:
df_clean = pd.DataFrame(structured_data)
df_clean

,Emp#,Name,Net Pay,Regular Hours,Rate,Regular Pay,ER SS,ER MC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,107,David R Cross nan nan nan nan nan Dept : Gener...,138.52,8.0,18.7500,150.00,9.30,2.18
2,231,Lorraine A Gunderson nan nan nan nan nan Dept ...,402.71,31.5,19.0000,598.50,37.11,8.68
3,169,Ellen J Hall nan nan nan nan nan Dept : Genera...,477.15,24.0,21.5278,516.67,32.03,7.49
4,268,Kenneth J Kelly III nan nan New Hire nan nan D...,946.82,40.0,26.0417,1041.67,64.58,15.10
5,247,Bryan P MacFadden nan nan nan nan nan Dept : G...,138.52,8.0,18.7500,150.00,9.30,2.18


In [18]:
df_clean['Name'][1]

'David R Cross nan nan nan nan nan Dept : General nan nan nan nan nan nan'

In [6]:
dfs[1]

,arnings,Hours* Rate,Current,Unnamed: 0,YTD,Taxes,Current.1,YTD Deductions,Unnamed: 1,Current.2,YTD.1,ER Taxes**,Current.3,YTD.2
0,Emp# 137,Sandra Manko,NaN,NaN,NaN,NaN,NaN,Dept : General,NaN,NaN,NaN,NaN,NaN,NaN
1,DirDep,Net Pay: 875.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Regular,40.00 29.1667,"1,166.67",NaN,"1,166.67",FWT,150.00,150.00 414(H),NaN,35.00,35.00,ER SS,72.33,72.33
3,NaN,NaN,NaN,NaN,NaN,SS W/H,72.33,72.33,NaN,NaN,NaN,ER MC,16.92,16.92
4,NaN,NaN,NaN,NaN,NaN,MC W/H,16.92,16.92,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NY State Tax,16.83,16.83,NaN,NaN,NaN,NaN,NaN,NaN
6,Employee Tot:,40.00,"1,166.67",NaN,"1,166.67",NaN,256.08,256.08,NaN,35.00,35.00,NaN,89.25,89.25
7,Emp# 234,Melissa J Olsen,NaN,NaN,NaN,NaN,NaN,Dept : General,NaN,NaN,NaN,NaN,NaN,NaN
8,DirDep,"Net Pay: 1,157.89",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Regular,74.00 18.018,"1,333.33",NaN,"1,333.33",FWT,120.00,120.00,NaN,NaN,NaN,ER SS,87.83,87.83


In [7]:
dfs[2]

,arnings,Hours* Rate,Current,Unnamed: 0,YTD,Taxes,Current.1,YTD Deductions,Unnamed: 1,Current.2,YTD.1,ER Taxes**,Current.3,YTD.2
0,Emp# 124,Brian Young,NaN,NaN,NaN,NaN,NaN,Dept : General,NaN,NaN,NaN,NaN,NaN,NaN
1,DirDep,Net Pay: 138.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Regular,8.00 18.75,150.00,NaN,150.00,FWT,0.00,0.00,NaN,NaN,NaN,ER SS,9.30,9.30
3,NaN,NaN,NaN,NaN,NaN,SS W/H,9.30,9.30,NaN,NaN,NaN,ER MC,2.18,2.18
4,NaN,NaN,NaN,NaN,NaN,MC W/H,2.18,2.18,NaN,NaN,NaN,NaN,NaN,NaN
5,Employee Tot:,8.00,150.00,NaN,150.00,NaN,11.48,11.48,NaN,0.00,0.00,NaN,11.48,11.48
6,Dept General,316.50,"7,036.01",NaN,"7,036.01",NaN,"1,146.12","1,146.12",NaN,43.38,43.38,NaN,538.26,538.26
7,Department : Town Outside Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Emp# 267,Jason J DeGiovanni,NaN,NaN,NaN,NaN,NaN,Dept : TOV,NaN,NaN,NaN,NaN,NaN,NaN
9,Chk# 16888,Net Pay: 280.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
pip install pdfplumber


     ---------------------------------------- 60.0/60.0 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 44.9 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 62.8 MB/s eta 0:00:00


In [120]:
print(text)

General
Department :
Emp# 107
David R Cross
Dept : General
DirDep
Regular
8.00
18.75
150.00
150.00
138.52
0.00
9.30
2.18
FWT
SS W/H
MC W/H
Net Pay:
ER SS
ER MC
9.30
2.18
9.30
2.18
0.00
9.30
2.18
Employee Tot:
8.00
150.00
150.00
11.48
0.00
11.48
11.48
0.00
11.48
Emp# 231
Lorraine A Gunderson
Dept : General
DirDep
Regular
31.50
19.00
598.50
598.50
402.71
150.00
37.11
8.68
FWT
SS W/H
MC W/H
Net Pay:
ER SS
ER MC
37.11
8.68
37.11
8.68
150.00
37.11
8.68
Employee Tot:
31.50
598.50
598.50
195.79
0.00
45.79
45.79
0.00
195.79
Emp# 169
Ellen J Hall
Dept : General
DirDep
Regular
24.00
21.5278
516.67
516.67
477.15
0.00
32.03
7.49
FWT
SS W/H
MC W/H
Net Pay:
ER SS
ER MC
32.03
7.49
32.03
7.49
0.00
32.03
7.49
Employee Tot:
24.00
516.67
516.67
39.52
0.00
39.52
39.52
0.00
39.52
Emp# 268
Kenneth J Kelly III
New Hire
Dept : General
DirDep
Regular
40.00
26.0417
1,041.67
1,041.67
946.82
0.00
64.58
15.10
15.17
FWT
SS W/H
MC W/H
NY State Tax
Net Pay:
ER SS
ER MC
64.58
15.10
64.58
15.10
0.00
64.58
15.10
15.17
E

In [119]:
import fitz  # PyMuPDF
import os
import re
import json

BASE_DIR = "Data"
INPUT_DIR = os.path.join(BASE_DIR, "input_files")
file_name = "2025-01-03 1 Payroll Set.pdf"

pdf_path = os.path.join(INPUT_DIR, file_name)
doc = fitz.open(pdf_path)

# === Extract full text ===
full_text = ""
for page in doc:
    full_text += page.get_text()

# === Use regex to extract employee chunks ===
# Matches from "Emp# <num>" to the next "Employee Tot:"
pattern = r"(Emp#.*?Employee Tot:.*?)(?=\nEmp#|\Z)"  # Non-greedy match between Emp# and next Emp# or end of text
employee_chunks = re.findall(pattern, full_text, flags=re.DOTALL)

# === Clean and save ===
cleaned_chunks = [chunk.strip() for chunk in employee_chunks if "Employee Tot:" in chunk]

output_path = os.path.join(BASE_DIR, "employee_chunks_raw_pdf.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cleaned_chunks, f, indent=2)

print(f"✅ Extracted {len(cleaned_chunks)} employee chunks and saved to {output_path}")


✅ Extracted 13 employee chunks and saved to Data\employee_chunks_raw_pdf.json
